In [ ]:
!pip install -q sunpy[map,net,timeseries]

In [ ]:
from datetime import datetime

from astropy.time import Time,TimeDelta
import astropy.units as u
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.timeseries import TimeSeries

# Setup support for plotting data with units
from astropy.visualization import quantity_support, time_support
quantity_support()
time_support()

# Search for some OMNI data
To search for different datasets you can use the form at https://cdaweb.gsfc.nasa.gov/index.html/

**Change the starttime, endtime and trange to plot data from desired date.**

In [ ]:
# Set up search parameters
starttime = datetime(2012, 1, 23, 0, 0, 0)
endtime = datetime(2012, 1, 24, 20, 30, 0)
trange = a.Time('2012/01/23 00:00:00', '2012/01/24 20:30:00')

dataset = a.cdaweb.Dataset('OMNI_COHO1HR_MERGED_MAG_PLASMA')
# dataset = a.cdaweb.Dataset('OMNI_HRO2_5MIN')

# Carry out the search
result = Fido.search(trange & dataset)
print(result)

# Download data

In [ ]:
downloaded_file_paths = Fido.fetch(result)
print(downloaded_file_paths)

In [ ]:
# The OMNI dataset contains some non-standard unit strings which we need
# to manually tell astropy about
u.add_enabled_units([u.def_unit('Deg K', u.K),
                     u.def_unit('Deg', u.deg),
                     u.def_unit('Re', u.R_earth)])
omni = TimeSeries(downloaded_file_paths, concatenate=True)
print(omni.columns)

# Plot Data
**Change xlim and individual ylims to change axis limites**

In [ ]:
time_support(format='plot_date', scale='utc')
xlim = Time(['2012-01-23T00:00:00', '2012-01-24T20:30:00'])
timearray = omni.time
fig, axs = plt.subplots(nrows=3, figsize=(10,12), sharex=True)
fig.suptitle('OMNI data from ' + timearray[0].to_value('iso',subfmt='date_hm') + ' to ' + timearray[-1].to_value('iso', subfmt='date_hm')) 

axs[0].plot(timearray, omni.quantity('V'))
axs[0].set_title(r'V')
axs[0].set_ylabel(r'km s$^{-1}$')
axs[0].set_ylim([350,700])

axs[1].plot(timearray, np.sqrt(omni.quantity('BT')**2 +omni.quantity('BR')**2 + omni.quantity('BN')**2))
axs[1].set_title(r'|B|')
axs[1].set_ylim([0,15])

axs[2].plot(timearray, omni.quantity('BN'))
axs[2].set_title(r'B$_z$')
axs[2].set_ylim([-10,15])
axs[2].set_xlabel('Time (UT)')
#set date format to be date_hms
date_form = DateFormatter("%Y-%m-%d %H:%M")
axs[2].xaxis.set_major_formatter(date_form)

for ax in axs:
    ax.grid()
    ax.set_xlim(xlim)

fig.autofmt_xdate() #automatically format times on x axis
fig.patch.set_alpha(1.0)